In [22]:
# import useful stuff
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import re
import numpy as np

from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score

# avoid undefined metric warning when calculating precision with 0 labels defined as 1
import warnings
warnings.filterwarnings('ignore')

### Data Analysis (complement)

In [2]:
# conyuemp and ult_fec_cli_1t represent edge cases, 
# respectively is spouse of employee or last date as primary customer (if was a primary customer and it is not)
# renta is another issue with almost 20% without renta information
# df_train.isnull().sum()/df_train.shape[0]

In [3]:
# showing cod_prov and nomprov have the same data, so one of them can be ignored
# pd.pivot_table(df_train[['cod_prov', 'nomprov']], index= ['cod_prov', 'nomprov'],
#              aggfunc=len)[:10]à

In [4]:
# fecha_dato onluy have few entries. treat it categorical
# df_train['fecha_dato'].value_counts()

In [5]:
# check if antiguedad and fecha_alta - fecha_dato hold the same info
#X['fecha_alta_month'] = X['fecha_alta']/30 + (35-30.0333333)
#np.corrcoef(X.ix[:100, 'fecha_alta_month'], X.ix[:100, 'antiguedad'])

# there are little variations on the data, but they are basically the same. I actually only need one

### Data transformations (from data analysis)

In [23]:
df_train = pd.read_csv('train_ver2.csv')

In [27]:
1.553689e+06

1553689.0

In [26]:
df_train['ncodpers'].describe()

count    1.364731e+07
mean     8.349042e+05
std      4.315650e+05
min      1.588900e+04
25%      4.528130e+05
50%      9.318930e+05
75%      1.199286e+06
max      1.553689e+06
Name: ncodpers, dtype: float64

In [46]:
a = df_train['ncodpers'].value_counts()[:-10]

In [55]:
a[a==11][:10]

1121701    11
1360215    11
1009355    11
1400613    11
244505     11
1361330    11
991576     11
1418682    11
746093     11
1336742    11
Name: ncodpers, dtype: int64

In [31]:
df_test = pd.read_csv('test_ver2.csv')

In [32]:
df_test['ncodpers'].describe()

count    9.296150e+05
mean     8.794566e+05
std      4.481569e+05
min      1.588900e+04
25%      4.833615e+05
50%      9.664250e+05
75%      1.264316e+06
max      1.553689e+06
Name: ncodpers, dtype: float64

In [35]:
# separate the labels
labels = []
for col in df_train.columns:
    if col[:4] == 'ind_' and col[-4:] == 'ult1':
        labels.append(col)


In [37]:
len(labels)

24

In [59]:
df_train.ix[df_train['ncodpers'] == 244505, ['fecha_dato', 'fecha_alta', 'segmento'] + labels[:12]]

,fecha_dato,fecha_alta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1
4192196,2015-07-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
5401965,2015-08-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
5787252,2015-09-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
6796618,2015-10-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
7562309,2015-11-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
8529556,2015-12-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
9149943,2016-01-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
10517798,2016-02-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
11130797,2016-03-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0
12106201,2016-04-28,2001-05-07,NaN,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
df_train.ix[df_train['ncodpers'] == 1121701, ['fecha_dato', 'fecha_alta', 'ind_actividad_cliente'] + labels[12:]]

,fecha_dato,fecha_alta,ind_actividad_cliente,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
4244503,2015-07-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
5129450,2015-08-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
6135767,2015-09-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
6327519,2015-10-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8031270,2015-11-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8438263,2015-12-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9720679,2016-01-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
10665732,2016-02-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11613214,2016-03-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
12271602,2016-04-28,2013-02-14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
def transform(df, fillna=False):
    """ This version transforms includes all variables, except if 100% redundant"""
    
    # remove cod_prov only, since it is redundant with nomprov
    for col in ['cod_prov']:
        del df[col]

    # convert dates for number of days between day and base day, which is given by fecha_dato
    # will be handled as numerical int
    date_vars = ['ult_fec_cli_1t']
    df[date_vars] = df[date_vars].convert_objects(convert_dates='coerce')
    df['ult_fec_cli_1t'] = (pd.to_datetime(df['fecha_dato'])-df['ult_fec_cli_1t']).astype('timedelta64[D]')

    # removed fecha_alta - redundant with antiguedad
    # df['fecha_alta'] = (pd.to_datetime(df['fecha_dato'])-df['fecha_alta']).astype('timedelta64[D]')
    del df['fecha_alta']
    
    # convert numerical vars to int
    numerical_vars = ['age', 'antiguedad', 'renta', 'ncodpers', 'ult_fec_cli_1t']
    df[numerical_vars] = df[numerical_vars].convert_objects(convert_numeric=True)

    # convert S/N to boolean and remaining to number
    boolean_vars = ['indfall', 'ind_actividad_cliente', 'ind_nuevo', 'indresi', 'indext', 
                    'tipodom', 'conyuemp']
    for var in ['indfall', 'indresi', 'indext', 'conyuemp']:
        df[var] = df[var] == 'S'
        
    # one hot encode categorical vars
    # fecha_dato have few options, to be handled as categorical 
    categorical_vars = ['segmento', 'sexo', 'tiprel_1mes', 'canal_entrada', 'nomprov', 
                        'pais_residencia', 'ind_empleado', 'fecha_dato']
    df = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, 
                       columns=categorical_vars, sparse=False, drop_first=False)
    
    # fill na with 0
    df = df.fillna(value=0)
            
    return df

In [7]:
def gen_data():
    # get data
    df = pd.read_csv('train_ver2.csv', nrows=1500000)
    
    # separate the labels
    labels = []
    for col in df.columns:
        if col[:4] == 'ind_' and col[-4:] == 'ult1':
            labels.append(col)

    # create X and y delete dataframe
    X = df[df.columns.difference(labels)]
    y = df[labels].fillna(value=0) # NAs in labels will be considered 0
    del df
    
    X = transform(X)
    y = y.loc[X.index]

    # order labels, to position the most popular products in front
    ordered_labels = []
    for label in labels:
        ordered_labels.append((label, (y[label] == 1).sum()))
    labels = [x for (x,y) in sorted(ordered_labels, key=lambda x:-x[1])]    
    
    return X,y, labels

In [8]:
X,y, labels = gen_data()


### Feature Selection

In [25]:
# let's do some feature selection
# In total I've got 351 features 
# most of them are categorical
# let's choose 3 labels are which more common, and work with them, to try and optimize features
for label in labels:
    print(label, y[label].sum()*1./X.shape[0])

ind_ahor_fin_ult1 0.000177
ind_aval_fin_ult1 3.9e-05
ind_cco_fin_ult1 0.749626
ind_cder_fin_ult1 0.000591
ind_cno_fin_ult1 0.105296
ind_ctju_fin_ult1 0.013623
ind_ctma_fin_ult1 0.009894
ind_ctop_fin_ult1 0.212486
ind_ctpp_fin_ult1 0.072079
ind_deco_fin_ult1 0.002158
ind_deme_fin_ult1 0.00315
ind_dela_fin_ult1 0.066881
ind_ecue_fin_ult1 0.106267
ind_fond_fin_ult1 0.027182
ind_hip_fin_ult1 0.009982
ind_plan_fin_ult1 0.014553
ind_pres_fin_ult1 0.004661
ind_reca_fin_ult1 0.072581
ind_tjcr_fin_ult1 0.066084
ind_valo_fin_ult1 0.039378
ind_viv_fin_ult1 0.006442
ind_nomina_ult1 0.071242
ind_nom_pens_ult1 0.079113
ind_recibo_ult1 0.166275


One of them stand out, ind_cco_fin_ult1, 75%. Second comes ind_ctop_fin_ult1 with 21% and 3rd ind_recibo_ult1, with 17%. Let's work with these three for feature selection

In [26]:
label = 'ind_cco_fin_ult1'
clf = DecisionTreeClassifier()
clf.fit(X,y[label])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [27]:
# 20 most important features per decision tree
list(sorted(zip(X.columns, clf.feature_importances_), key=lambda x:-x[1]))

[('ncodpers', 0.23925455512523605),
 ('renta', 0.19618125184267854),
 ('age', 0.15566533417947886),
 ('canal_entrada_KHE', 0.092799750530079592),
 ('antiguedad', 0.076423282471976675),
 ('nomprov_MADRID', 0.014343319778113714),
 ('ind_actividad_cliente', 0.013040959612069511),
 ('canal_entrada_KAT', 0.0093977732953335197),
 ('nomprov_BARCELONA', 0.0090709750147857805),
 ('sexo_V', 0.008480773190250028),
 ('sexo_H', 0.0080663943388401896),
 ('canal_entrada_KFC', 0.0079991961693934421),
 ('nomprov_VALENCIA', 0.0065679641282231629),
 ('indext', 0.0062142021669815885),
 ('nomprov_SEVILLA', 0.0060595865209062214),
 ('nomprov_MALAGA', 0.0047956371343746062),
 ('fecha_dato_2015-02-28', 0.0045810236553350397),
 ('fecha_dato_2015-01-28', 0.0044538343317748362),
 ('segmento_02 - PARTICULARES', 0.0044413874144089783),
 ('tiprel_1mes_A', 0.0042782212311818129),
 ('nomprov_ZARAGOZA', 0.0042225388888579756),
 ('canal_entrada_KFA', 0.0041101243504189202),
 ('nomprov_ALICANTE', 0.0041052158590681662),

In [28]:
# 20 most important features per selectKbest, and their p-value which is relevant
selector = SelectKBest(k=20)
selector.fit(X,y[label])
list(sorted(zip(X.columns, selector.scores_, selector.pvalues_), key=lambda x:-x[1]))

[('canal_entrada_KHE', 102292.26125632203, 0.0),
 ('segmento_03 - UNIVERSITARIO', 90490.039558362623, 0.0),
 ('ncodpers', 66156.704967162485, 0.0),
 ('segmento_02 - PARTICULARES', 45668.446242552563, 0.0),
 ('canal_entrada_KAT', 17901.791005238305, 0.0),
 ('nomprov_MADRID', 17601.747192783132, 0.0),
 ('age', 17054.519040127234, 0.0),
 ('tiprel_1mes_I', 16133.508427903529, 0.0),
 ('tiprel_1mes_A', 10981.050792527869, 0.0),
 ('tipodom', 10746.601898096182, 0.0),
 ('indrel_1mes', 10380.216892595772, 0.0),
 ('ind_empleado_N', 10158.619892102673, 0.0),
 ('fecha_dato_2015-01-28', 8275.1779209089727, 0.0),
 ('fecha_dato_2015-02-28', 8275.1779209089727, 0.0),
 ('segmento_01 - TOP', 6369.3679074336787, 0.0),
 ('indresi', 6146.5951201432099, 0.0),
 ('pais_residencia_ES', 6146.5951201432099, 0.0),
 ('canal_entrada_KFC', 5225.4327914087053, 0.0),
 ('ind_actividad_cliente', 3002.5331821983145, 0.0),
 ('nomprov_MURCIA', 2814.1093240785149, 0.0),
 ('canal_entrada_KFA', 2199.8561237798317, 0.0),
 ('se

There are a lot of coincidences between both lists. In the categorical variables, we have canal_entrada_KHE, segmento Universitarios. In the numerical variables, there is age, ncodepers ( a sequential id that will tell which customer came before) and antiguedad, which is also represented by fecha_alta and fecha_alta_month (redundancy to be removed). For the big categorical variables, we can see that province > canal_entrada > country of residence.


### Benchmark

In [29]:
# assign all 0s and 1s, and check accuracy
# benchmark

label = 'ind_cco_fin_ult1'
pred0 = np.zeros(y.shape[0])
pred1 = np.zeros(y.shape[0])

print("Accuracy all 1s: {:.2f}".format(accuracy_score(y[label], pred1)))
print("Accuracy all 0s: {:.2f}".format(accuracy_score(y[label], pred0)))

y[label].value_counts()

Accuracy all 1s: 0.25
Accuracy all 0s: 0.25


1    749626
0    250374
Name: ind_cco_fin_ult1, dtype: int64

### Results with and without feature selection

Let's try predict with and without feature selection to get a general understanding whether it will help the classifier or not


In [30]:
# without:
clf = DecisionTreeClassifier()
score = cross_val_score(clf, X, y[label])
print("Mean score: {:.2f}, Std: {:.2f}".format(score.mean(), score.std()))

Mean score: 0.53, Std: 0.16


In [31]:
# with:
label = 'ind_cco_fin_ult1'
for i in range(1,10,2):
    selector = SelectKBest(k=i)
    X_selected = selector.fit_transform(X, y[label])
    clf = DecisionTreeClassifier()
    score = cross_val_score(clf, X_selected, y[label])
    print("{} features: Mean score: {:.2f}, Std: {:.2f}".format(i, score.mean(), score.std()))

1 features: Mean score: 0.75, Std: 0.00
3 features: Mean score: 0.90, Std: 0.07
5 features: Mean score: 0.90, Std: 0.07
7 features: Mean score: 0.90, Std: 0.07
9 features: Mean score: 0.90, Std: 0.07


Less features makes a huge difference in the results. The problem is for each product, we might deal with different features. So it is best to select them before training each classifier.

It seems 3 features are enough to drive accuracy to 90%. Performance starts to drop after 12 features. This should reflect on the feature transformation as well - most of the variabiity will be encoded in a few components.

### Feature Transformation

Instead of selecting features, let's try to use as most as possible by transforming the features into principal components and capturing their variability in a small number of features. 

First we will need to scale the features, then apply a principal component analysis and extract the main features. We will aim at getting the components that explain up to 80% in the variability.

In [32]:
# scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [33]:
pca = PCA(n_components=20)
X_pc = pca.fit_transform(X_scaled, y[label])

# Can I get a good model with 12 components, or 80% explained variance?
clf = DecisionTreeClassifier()
score = cross_val_score(clf, X_pc, y[label])
print("Mean score: {:.2f}, Std: {:.2f}".format(score.mean(), score.std()))


Mean score: 0.46, Std: 0.19


In [34]:
for i in range(1,10):
    print(i, pca.explained_variance_ratio_[:i].sum())

1 0.240505022952
2 0.373810519771
3 0.490166483124
4 0.580849256425
5 0.653848257105
6 0.705532844255
7 0.73015535148
8 0.751299274439
9 0.770741410394


In [35]:
# try different number of components
for i in [2,5,10,50,100]:
    
    pca = PCA(n_components=i)
    X_pc = pca.fit_transform(X_scaled, y[label])

    clf = DecisionTreeClassifier()
    score = cross_val_score(clf, X_pc, y[label])
    print("Mean score: {:.2f}, Std: {:.2f}".format(score.mean(), score.std()))


Mean score: 0.50, Std: 0.17
Mean score: 0.46, Std: 0.19
Mean score: 0.47, Std: 0.19
Mean score: 0.48, Std: 0.19
Mean score: 0.47, Std: 0.19


### Predicting with feature selection in pipeline

After all required corvertions have been made, I can make a first shot at predicting. First question we need to ask is, what I'm a predicting?

I'm predicting comsuption of a certain product. I have a total of 24 booleans that will tell whether or not this customer consumed this product. These are my labels for a One vs All classification model.



In [9]:
X,y,labels=gen_data()

In [10]:
# upload test data
X_test = pd.read_csv('test_ver2.csv')

# initialize results
report = pd.DataFrame(X_test['ncodpers'])
classif_results = {}

# prepare test data for classifer
X_test = transform(X_test, fillna=True)


In [11]:
# X_test should only have columns that are also in X (needed due to one-hot encoding)
paired_columns = list(set(X_test.columns).intersection(X.columns))
X_test = X_test[paired_columns]
X = X[paired_columns]

In [12]:
# predict each product with a different clssifer
for label in labels:
    if len(y[label].value_counts()) != 1:
        # select 6 features
        selector = SelectKBest(k=6)
        X_selected = selector.fit_transform(X, y[label])
        
        # classify
        clf = DecisionTreeClassifier()
        clf.fit(X_selected, y[label])
        X_test_selected = selector.transform(X_test)
        classif_results[label] = clf.predict(X_test_selected)

In [13]:
# clean memory
del X
del y
del X_test

In [14]:
# transform results to expected output
fn_name_labels = lambda label, pred: list(map(lambda x: label if x else '', pred))
cf_list = [fn_name_labels(k,v) for k,v in classif_results.items()]

# concatenate results
fn_join_columns = lambda x:re.sub('\s+', ' ', ' '.join(x)).strip()

# add new column added products in report
report['added_products'] = list(map(fn_join_columns, zip(*cf_list)))

In [15]:
report.ix[0, 'added_products']

'ind_dela_fin_ult1 ind_ctpp_fin_ult1 ind_cco_fin_ult1 ind_valo_fin_ult1'

In [16]:
report.to_csv('round3b.csv', header=True, index=False)